# CASE WHEN
### Use case: creating categorization from a numerical column

## Set up

In [ ]:
import os
import pandas as pd

In [2]:
#os.getcwd()
os.chdir('/Users/yupinghe/Downloads')

In [3]:
print(spark.version)
print(type(sc)) # default spark context
print(type(spark)) # default spark session
print(sc.master) # default master

2.3.3
<class 'pyspark.context.SparkContext'>
<class 'pyspark.sql.session.SparkSession'>
local[2]


In [4]:
d = pd.read_csv('applications data.csv')

In [5]:
spark.createDataFrame(d).createOrReplaceTempView('applications')

## Wide format

In [21]:
query2 = \
"SELECT \
SUM(CASE WHEN to_date(string(date),'yyyyMMdd') >= '2016-01-01' AND to_date(string(date),'yyyyMMdd') <= '2016-03-31' THEN 1 ELSE 0 END) AS Q1, \
SUM(CASE WHEN to_date(string(date),'yyyyMMdd') >= '2016-04-01' AND to_date(string(date),'yyyyMMdd') <= '2016-06-30' THEN 1 ELSE 0 END) AS Q2, \
SUM(CASE WHEN to_date(string(date),'yyyyMMdd') >= '2016-07-01' AND to_date(string(date),'yyyyMMdd') <= '2016-09-30' THEN 1 ELSE 0 END) AS Q3, \
SUM(CASE WHEN to_date(string(date),'yyyyMMdd') >= '2016-10-01' AND to_date(string(date),'yyyyMMdd') <= '2016-12-31' THEN 1 ELSE 0 END) AS Q4 \
FROM applications"

spark.sql(query2).show()

+------+------+------+------+
|    Q1|    Q2|    Q3|    Q4|
+------+------+------+------+
|246862|249633|252147|251358|
+------+------+------+------+



## Long format

In [25]:
query1 = \
"SELECT Quarter, count(*) AS Count \
 FROM (SELECT \
        CASE \
        WHEN to_date(string(date), 'yyyyMMdd') >= '2016-01-01' AND to_date(string(date), 'yyyyMMdd') <= '2016-03-31' THEN 'Q1' \
        WHEN to_date(string(date), 'yyyyMMdd') >= '2016-04-01' AND to_date(string(date), 'yyyyMMdd') <= '2016-06-30' THEN 'Q2' \
        WHEN to_date(string(date), 'yyyyMMdd') >= '2016-07-01' AND to_date(string(date), 'yyyyMMdd') <= '2016-09-30' THEN 'Q3' \
        WHEN to_date(string(date), 'yyyyMMdd') >= '2016-10-01' AND to_date(string(date), 'yyyyMMdd') <= '2016-12-31' THEN 'Q4' \
        END AS Quarter  \
       FROM applications) a\
 GROUP BY Quarter"

spark.sql(query1).show()

+-------+------+
|Quarter| Count|
+-------+------+
|     Q2|249633|
|     Q1|246862|
|     Q3|252147|
|     Q4|251358|
+-------+------+



## Close out

In [29]:
spark.catalog.dropTempView("applications")
spark.catalog.listTables()

[]